In [4]:
!pip install -r requirements.txt

In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import plotly.graph_objs as go

# Set a seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define stock data retrieval parameters
start_date = '2000-01-01'
end_date = '2024-02-01'
stock_symbol = 'MSFT'

# Download stock data
data = yf.download(stock_symbol, start=start_date, end=end_date)

# Reset index and drop missing values
data.reset_index(inplace=True)
data.dropna(inplace=True)

# Calculate moving averages
data['MA100'] = data['Close'].rolling(100).mean()
data['MA200'] = data['Close'].rolling(200).mean()

# Prepare dataset for training
def create_dataset(dataset, look_back=100):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

# Use close, MA100, and MA200 for training
features_considered = ['Close', 'MA100', 'MA200']
features = data[features_considered]
features = features.dropna()

# Normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# Split data into train and test sets
train_size = int(len(scaled_features) * 0.8)
train_data = scaled_features[:train_size]
test_data = scaled_features[train_size:]

# Prepare the data for the LSTM model
look_back = 100
x_train, y_train = create_dataset(train_data, look_back)
x_test, y_test = create_dataset(test_data, look_back)

# Reshape input to be [samples, time steps, features]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], len(features_considered)))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], len(features_considered)))

# Model definition with an Input layer, assuming you want to follow the warning's advice
model = Sequential([
    Input(shape=(x_train.shape[1], x_train.shape[2])),  # Explicit Input layer
    LSTM(50, return_sequences=True),
    LSTM(50),
    Dense(25),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# EarlyStopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit the model (use a validation split to monitor validation loss)
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Predictions on test set
y_pred = model.predict(x_test)

# Inverse transform to original scale
y_pred_inverse = scaler.inverse_transform(np.concatenate((y_pred, np.zeros((y_pred.shape[0], len(features_considered)-1))), axis=1))[:,0]
y_test_inverse = scaler.inverse_transform(np.concatenate((y_test.reshape(-1,1), np.zeros((y_test.shape[0], len(features_considered)-1))), axis=1))[:,0]

# Calculate evaluation metrics
mae = mean_absolute_error(y_test_inverse, y_pred_inverse)
mse = mean_squared_error(y_test_inverse, y_pred_inverse)
print("LSTM Model Mean Absolute Error (MAE):", mae)
print("LSTM Model Mean Squared Error (MSE):", mse)

# Adjusting date alignment for plotting
test_dates = data['Date'][train_size+look_back:]

# Plot original vs predicted prices using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_dates, y=y_test_inverse.flatten(), mode='lines', name='Original Price'))
fig.add_trace(go.Scatter(x=test_dates, y=y_pred_inverse.flatten(), mode='lines', name='Predicted Price'))
fig.update_layout(title='LSTM Model - Original vs Predicted Prices',
                   xaxis_title='Date',
                   yaxis_title='Price')
fig.show()


[*********************100%%**********************]  1 of 1 completed


Epoch 1/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 16s 90ms/step - loss: 1.8639e-04 - val_loss: 7.9659e-05
Epoch 2/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 9s 79ms/step - loss: 1.2546e-05 - val_loss: 3.0436e-05
Epoch 3/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - loss: 9.4707e-06 - val_loss: 2.7830e-05
Epoch 4/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - loss: 7.7310e-06 - val_loss: 2.8948e-05
Epoch 5/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - loss: 7.1689e-06 - val_loss: 4.2172e-05
Epoch 6/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - loss: 6.9272e-06 - val_loss: 4.7718e-05
Epoch 7/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 9s 78ms/step - loss: 7.1648e-06 - val_loss: 3.8278e-05
Epoch 8/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 11s 85ms/step - loss: 6.9592e-06 - val_loss: 3.0372e-05
Epoch 9/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 11s 94ms/step - loss: 6.6415e-06 - val_loss: 2.4839e-05
Epoch 10/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - loss: 6.2900e-06 - val_loss: 2.1570e-05
Epoch 11/100
115/115 ━━━━━━━━━━